In [1]:
import pandas as pd
import helpers
import json
import numpy as np
import random
import math
import collections

In [152]:
items = helpers.load_items()
item_list = list(items.keys())

In [142]:
TEST = False
interactions = helpers.load_interactions_df() if not TEST else helpers.load_interactions_test_df() 

In [143]:
interactions

,user_id,item_id,event_type,event_timestamp,target
0,0,1786148,view,2019-10-19T11:25:42.444-0400,1748830
1,0,1786148,view,2019-10-19T11:25:57.487-0400,1748830
2,0,RELOGIO SMARTWATCH,search,2019-10-19T11:26:07.063-0400,1748830
3,0,1615991,view,2019-10-19T11:27:26.879-0400,1748830
4,0,1615991,view,2019-10-19T11:28:36.558-0400,1748830
...,...,...,...,...,...
11999161,413162,1348242,view,2019-10-04T08:34:56.361-0400,1212472
11999162,413162,792798,view,2019-10-07T10:49:32.217-0400,1212472
11999163,413162,258196,view,2019-10-07T10:52:21.825-0400,1212472
11999164,413162,12716,view,2019-10-07T10:53:07.399-0400,1212472


In [171]:
if not TEST:
    bought_items = interactions.groupby('user_id')['target'].unique().apply(list).to_dict()
    bought_items

In [172]:
def get_recommendations(x):
    df = x[x['event_type'] != 'search']
    preds = df['item_id'].tolist()
    if len(preds) < 10:
        #domains = [items[x]['domain_id'] for x in preds.iterrows()]
        #counter = collection.Counter(domains)
        k = 10 - len(preds)
        preds = [] + preds
    return preds[:10]

viewed = (interactions if TEST else interactions).groupby('user_id').apply(get_recommendations).to_dict()
viewed

{0: ['1786148',
  '1786148',
  '1615991',
  '1615991',
  '1615991',
  '1615991',
  '1615991',
  '1615991',
  '1615991',
  '1615991'],
 1: [1444781,
  885610,
  '643652',
  '1156086',
  '1943604',
  '206667',
  '1282813',
  '228737',
  '228737',
  '228737'],
 2: [1563608,
  61854,
  1693380,
  1065600,
  711841,
  91338,
  1865229,
  741001,
  '248595',
  '248595'],
 3: [1537172,
  1661845,
  813161,
  '505541',
  '505541',
  '505541',
  '505541',
  '505541',
  '1230082',
  '937557'],
 4: ['130440',
  '130440',
  '2049207',
  '2049207',
  '2049207',
  '277311',
  '1476099',
  '2082794',
  '1476099',
  '1376286'],
 5: [423182,
  918934,
  735662,
  95844,
  1709247,
  368282,
  '2029385',
  '794504',
  '1135701',
  '1135701'],
 6: [328076,
  636838,
  286448,
  1914793,
  1284474,
  93400,
  1912999,
  '400631',
  '1998468',
  '1156202'],
 7: [1758631,
  829481,
  260559,
  20276,
  685994,
  1333058,
  1677498,
  344542,
  1485739,
  '375282'],
 8: [285247,
  1525971,
  1816829,
  11275

In [173]:
def relevance(item, target):
    if item == target:
        return 12
    elif items[item]['domain_id'] == items[target]['domain_id']:
        return 1
    return 0

def dcg(r, target):
    dcg = sum(relevance(r[i], target) / math.log(i + 2) for i in range(len(r)))
    return dcg# / perfect_score

def get_perfect():
    perfect = [12, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    assert len(perfect) == 10
    return sum(perfect[i] / math.log(i + 2) for i in range(len(perfect)))

In [174]:
if not TEST:
    sum_dcg = 0
    for x in viewed.keys():
        sum_dcg += dcg([int(w) for w in viewed[x]], bought_items[x][0])

    print(sum_dcg / (get_perfect() * len(viewed.keys())))

0.21114882419104541


In [137]:
submit = pd.DataFrame(viewed)
submit

,0,1,2,3,4,5,6,7,8,9,...,177060,177061,177062,177063,177064,177065,177066,177067,177068,177069
0,1572239,831243,1986443,1542656,990519,1729849,232562,104535,1695970,1984730,...,1809463,1158631,2012254,1545230,647286,998504,1152672,1423904,313289,1810757
1,1572239,1125393,991246,820887,1687165,1360482,1960672,1229404,378293,1361154,...,1317489,1646843,1681497,1545230,1423914,508454,1907343,273237,1895730,1559210
2,1572239,831243,1660615,281288,1185022,417142,1480574,1751720,378293,1984730,...,1334397,8726,1119097,2021996,1311881,1534892,1384867,2093769,702417,1559210
3,1572239,943786,283306,592703,176606,417142,773952,1521031,378293,253933,...,276558,1266546,343754,459873,1311881,1165244,765481,136713,1117363,1559210
4,1572239,943786,1425647,663106,171250,417142,1330345,280469,378293,1984730,...,1440221,8726,593466,1545230,1625436,646074,1763050,1221968,406725,1507800
5,1572239,943786,1362449,1001461,1398419,2012901,1246530,1753298,1792353,86082,...,601090,517342,840964,1545230,1625436,1584422,1191685,246147,1093081,152660
6,1572239,943786,167018,2088287,1398419,785074,684728,861597,1792353,1587422,...,1854085,1266546,1776750,1919129,1625436,1836040,1183025,2033823,1462592,1428627
7,1572239,943786,539599,803666,1398419,2012901,1203834,788503,1254344,702406,...,755000,325698,163694,1919129,1625436,1523916,524786,2033823,1155931,1841925
8,1572239,831243,1219935,463663,1398419,2012901,1203834,568643,903401,1587422,...,736639,136240,1721582,1919129,1625436,1711522,1651522,1061535,1803169,486214
9,1194894,48097,1614538,241502,1398419,139969,1725528,931987,943510,1587422,...,791232,136240,1721582,1919129,1625436,24604,331472,1061535,1803169,389656


In [138]:
submit.transpose()

,0,1,2,3,4,5,6,7,8,9
0,1572239,1572239,1572239,1572239,1572239,1572239,1572239,1572239,1572239,1194894
1,831243,1125393,831243,943786,943786,943786,943786,943786,831243,48097
2,1986443,991246,1660615,283306,1425647,1362449,167018,539599,1219935,1614538
3,1542656,820887,281288,592703,663106,1001461,2088287,803666,463663,241502
4,990519,1687165,1185022,176606,171250,1398419,1398419,1398419,1398419,1398419
...,...,...,...,...,...,...,...,...,...,...
177065,998504,508454,1534892,1165244,646074,1584422,1836040,1523916,1711522,24604
177066,1152672,1907343,1384867,765481,1763050,1191685,1183025,524786,1651522,331472
177067,1423904,273237,2093769,136713,1221968,246147,2033823,2033823,1061535,1061535
177068,313289,1895730,702417,1117363,406725,1093081,1462592,1155931,1803169,1803169


In [170]:
submit.transpose().to_csv('submit.csv', index=False, header=False)